# Visualization and Plotting

This notebook demonstrates Astrora's comprehensive visualization capabilities for orbital mechanics.

**Topics covered:**
- 2D orbit plotting (matplotlib)
- 3D interactive visualization (plotly)
- Ground track plotting
- Orbit animations (2D and 3D)
- Multiple orbit comparisons
- Custom styling and themes

**Prerequisites:** Basic understanding of orbits from previous notebooks

## Setup

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from astropy import units as u
from astrora import Orbit, bodies
from astrora.plotting import plot_ground_track, plot_orbit_2d, plot_orbit_3d
from astrora.plotting.animation import animate_orbit, animate_orbit_3d

%matplotlib inline
plt.rcParams['figure.figsize'] = (12, 10)

print("✓ Imports successful!")

## 1. Basic 2D Orbit Plotting

Start with simple 2D projections of orbits:

In [ ]:
# Create ISS orbit
iss = Orbit.from_classical(
    bodies.Earth,
    a=6778 * u.km,
    ecc=0.0005 * u.one,
    inc=51.6 * u.deg,
    raan=0 * u.deg,
    argp=0 * u.deg,
    nu=0 * u.deg
)

# Plot using built-in function
fig, ax = plot_orbit_2d(iss, color='red', linewidth=2, label='ISS Orbit')
ax.set_title('International Space Station Orbit', fontsize=14, fontweight='bold')
plt.show()

print(f"ISS Orbital Parameters:")
print(f"  Altitude: {(iss.a - bodies.Earth.R).to(u.km):.1f}")
print(f"  Period: {iss.period.to(u.minute):.1f}")
print(f"  Inclination: {iss.inc.to(u.deg):.1f}")

### Multiple Orbits Comparison

In [ ]:
# Create different orbit types
orbits = [
    {
        'name': 'LEO',
        'orbit': Orbit.from_classical(bodies.Earth, a=6778*u.km, ecc=0.0*u.one,
                                      inc=28.5*u.deg, raan=0*u.deg, argp=0*u.deg, nu=0*u.deg),
        'color': 'blue'
    },
    {
        'name': 'MEO',
        'orbit': Orbit.from_classical(bodies.Earth, a=20200*u.km, ecc=0.0*u.one,
                                      inc=55*u.deg, raan=0*u.deg, argp=0*u.deg, nu=0*u.deg),
        'color': 'green'
    },
    {
        'name': 'GEO',
        'orbit': Orbit.from_classical(bodies.Earth, a=42164*u.km, ecc=0.0*u.one,
                                      inc=0*u.deg, raan=0*u.deg, argp=0*u.deg, nu=0*u.deg),
        'color': 'red'
    },
]

# Create figure with multiple orbits
fig = plt.figure(figsize=(14, 14))
ax = fig.add_subplot(111, projection='3d')

# Plot Earth
u_sphere = np.linspace(0, 2 * np.pi, 50)
v_sphere = np.linspace(0, np.pi, 50)
x_earth = 6371 * np.outer(np.cos(u_sphere), np.sin(v_sphere))
y_earth = 6371 * np.outer(np.sin(u_sphere), np.sin(v_sphere))
z_earth = 6371 * np.outer(np.ones(np.size(u_sphere)), np.cos(v_sphere))
ax.plot_surface(x_earth, y_earth, z_earth, color='lightblue', alpha=0.5)

# Plot each orbit
for orb_data in orbits:
    orbit = orb_data['orbit']
    name = orb_data['name']
    color = orb_data['color']

    # Propagate orbit
    theta = np.linspace(0, 2*np.pi, 100)
    positions = []
    for angle in theta:
        orb = Orbit.from_classical(
            orbit.attractor, orbit.a, orbit.ecc, orbit.inc,
            orbit.raan, orbit.argp, angle * u.rad
        )
        positions.append(orb.r.to_value('km'))
    positions = np.array(positions)

    ax.plot(positions[:, 0], positions[:, 1], positions[:, 2],
            color=color, linewidth=2, label=name)

ax.set_xlabel('X (km)', fontsize=11)
ax.set_ylabel('Y (km)', fontsize=11)
ax.set_zlabel('Z (km)', fontsize=11)
ax.set_title('Orbital Regimes Comparison', fontsize=14, fontweight='bold')
ax.legend(loc='upper left', fontsize=12)
ax.set_box_aspect([1,1,1])
plt.tight_layout()
plt.show()

# Print orbital periods
print("Orbital Periods:")
for orb_data in orbits:
    period = orb_data['orbit'].period.to(u.hour)
    print(f"  {orb_data['name']:10s}: {period:.2f}")

## 2. 3D Interactive Visualization with Plotly

Plotly enables interactive 3D visualization that you can rotate, zoom, and explore:

In [ ]:
# Create a highly elliptical orbit (Molniya-type)
molniya = Orbit.from_classical(
    bodies.Earth,
    a=26554 * u.km,
    ecc=0.74 * u.one,
    inc=63.4 * u.deg,  # Critical inclination (no apsidal precession)
    raan=0 * u.deg,
    argp=270 * u.deg,  # Apogee over northern hemisphere
    nu=0 * u.deg
)

# Plot using built-in 3D function
fig = plot_orbit_3d(molniya, color='red', name='Molniya Orbit')
fig.update_layout(
    title='Molniya Orbit - 3D Interactive View',
    scene=dict(
        xaxis_title='X (km)',
        yaxis_title='Y (km)',
        zaxis_title='Z (km)',
        aspectmode='data'
    ),
    width=900,
    height=700
)
fig.show()

print(f"Molniya Orbit Characteristics:")
print(f"  Perigee: {molniya.r_p.to(u.km):.0f}")
print(f"  Apogee: {molniya.r_a.to(u.km):.0f}")
print(f"  Period: {molniya.period.to(u.hour):.2f} (≈12 hours for ground track repeat)")
print(f"  Inclination: {molniya.inc.to(u.deg):.1f} (critical for zero apsidal precession)")

## 3. Ground Track Plotting

Ground tracks show the sub-satellite point trajectory on Earth's surface:

In [ ]:
# ISS ground track
fig, ax = plot_ground_track(
    iss,
    duration=90 * u.minute,  # One orbit
    n_points=500,
    color='red',
    linewidth=2
)
ax.set_title('ISS Ground Track (One Orbit)', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

print("ISS Ground Track Features:")
print(f"  Maximum latitude: ±{iss.inc.to(u.deg).value:.1f}° (= inclination)")
print(f"  Orbit period: {iss.period.to(u.minute):.1f}")
print(f"  Ground track repeats every ~3 days")

### Sun-Synchronous Orbit Ground Track

In [ ]:
# Sun-synchronous orbit (typical Earth observation satellite)
sso = Orbit.from_classical(
    bodies.Earth,
    a=7078 * u.km,  # ~700 km altitude
    ecc=0.001 * u.one,
    inc=98.2 * u.deg,  # Retrograde, sun-synchronous
    raan=0 * u.deg,
    argp=0 * u.deg,
    nu=0 * u.deg
)

# Plot ground track over 24 hours
fig, ax = plot_ground_track(
    sso,
    duration=24 * u.hour,
    n_points=2000,
    color='blue',
    linewidth=1.5
)
ax.set_title('Sun-Synchronous Orbit Ground Track (24 hours)', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

n_orbits = (24 * u.hour / sso.period).decompose()
print(f"Sun-Synchronous Orbit:")
print(f"  Altitude: {(sso.a - bodies.Earth.R).to(u.km):.0f}")
print(f"  Period: {sso.period.to(u.minute):.1f}")
print(f"  Orbits per day: {n_orbits:.1f}")
print(f"  Coverage: Passes over same location at same local solar time each day")

## 4. Orbit Animation (2D)

Animate orbits to visualize satellite motion over time:

In [ ]:
# Create GTO (Geostationary Transfer Orbit)
gto = Orbit.from_classical(
    bodies.Earth,
    a=24378 * u.km,
    ecc=0.73 * u.one,
    inc=28.5 * u.deg,
    raan=0 * u.deg,
    argp=0 * u.deg,
    nu=0 * u.deg
)

print("Creating 2D animation of GTO...")
print("Note: Animation will save to file and can be displayed in Jupyter")

# Create animation (saves to GIF)
anim = animate_orbit(
    gto,
    duration=1.0,  # One full period
    n_frames=100,
    show_trail=True,
    fps=20,
    output_file='gto_animation.gif'
)

print("✓ Animation created: gto_animation.gif")
print(f"\nGTO Properties:")
print(f"  Perigee: {gto.r_p.to(u.km):.0f}")
print(f"  Apogee: {gto.r_a.to(u.km):.0f}")
print(f"  Period: {gto.period.to(u.hour):.2f}")

## 5. 3D Interactive Animation

Create interactive 3D animations using Plotly:

In [ ]:
# Create polar orbit
polar = Orbit.from_classical(
    bodies.Earth,
    a=7178 * u.km,
    ecc=0.0 * u.one,
    inc=90 * u.deg,
    raan=0 * u.deg,
    argp=0 * u.deg,
    nu=0 * u.deg
)

print("Creating 3D interactive animation...")

# Create 3D animation (saves to HTML)
fig = animate_orbit_3d(
    polar,
    duration=1.0,  # One period
    n_frames=60,
    show_trail=True,
    output_file='polar_orbit_3d.html'
)

# Display inline
fig.show()

print("✓ 3D animation created: polar_orbit_3d.html")
print(f"\nPolar Orbit:")
print(f"  Altitude: {(polar.a - bodies.Earth.R).to(u.km):.0f}")
print(f"  Inclination: {polar.inc.to(u.deg):.0f}")
print(f"  Period: {polar.period.to(u.minute):.1f}")

## 6. Custom Styling and Themes

Create publication-quality plots with custom styling:

In [ ]:
# Dark theme plot
plt.style.use('dark_background')

fig = plt.figure(figsize=(14, 14))
ax = fig.add_subplot(111, projection='3d')

# Earth with dark theme
u_sphere = np.linspace(0, 2 * np.pi, 50)
v_sphere = np.linspace(0, np.pi, 50)
x_earth = 6371 * np.outer(np.cos(u_sphere), np.sin(v_sphere))
y_earth = 6371 * np.outer(np.sin(u_sphere), np.sin(v_sphere))
z_earth = 6371 * np.outer(np.ones(np.size(u_sphere)), np.cos(v_sphere))
ax.plot_surface(x_earth, y_earth, z_earth, color='#1f77b4', alpha=0.3)

# Multiple orbits with custom colors
orbits_custom = [
    {'orbit': iss, 'color': '#ff6b6b', 'label': 'ISS', 'linewidth': 3},
    {'orbit': molniya, 'color': '#4ecdc4', 'label': 'Molniya', 'linewidth': 2.5},
    {'orbit': polar, 'color': '#ffe66d', 'label': 'Polar', 'linewidth': 2},
]

for orb_data in orbits_custom:
    orbit = orb_data['orbit']
    theta = np.linspace(0, 2*np.pi, 150)
    positions = []
    for angle in theta:
        orb = Orbit.from_classical(
            orbit.attractor, orbit.a, orbit.ecc, orbit.inc,
            orbit.raan, orbit.argp, angle * u.rad
        )
        positions.append(orb.r.to_value('km'))
    positions = np.array(positions)

    ax.plot(positions[:, 0], positions[:, 1], positions[:, 2],
            color=orb_data['color'], linewidth=orb_data['linewidth'],
            label=orb_data['label'], alpha=0.9)

# Styling
ax.set_xlabel('X (km)', fontsize=12, color='white')
ax.set_ylabel('Y (km)', fontsize=12, color='white')
ax.set_zlabel('Z (km)', fontsize=12, color='white')
ax.set_title('Multiple Satellite Orbits - Dark Theme',
             fontsize=16, fontweight='bold', color='white', pad=20)
ax.legend(loc='upper left', fontsize=12, framealpha=0.8)
ax.set_facecolor('#0d1117')
ax.grid(True, alpha=0.2)
ax.set_box_aspect([1,1,1])

plt.tight_layout()
plt.show()

# Reset style
plt.style.use('default')

print("✓ Dark theme visualization complete")

## 7. Comparing Transfer Orbits

Visualize different transfer strategies:

In [ ]:
# LEO and GEO
r_leo = 6778e3
r_geo = 42164e3

leo = Orbit.from_classical(bodies.Earth, a=r_leo*u.m, ecc=0.0*u.one,
                           inc=0*u.deg, raan=0*u.deg, argp=0*u.deg, nu=0*u.deg)

geo = Orbit.from_classical(bodies.Earth, a=r_geo*u.m, ecc=0.0*u.one,
                           inc=0*u.deg, raan=0*u.deg, argp=0*u.deg, nu=0*u.deg)

# Hohmann transfer
a_hohmann = (r_leo + r_geo) / 2
ecc_hohmann = (r_geo - r_leo) / (r_geo + r_leo)
hohmann = Orbit.from_classical(bodies.Earth, a=a_hohmann*u.m, ecc=ecc_hohmann*u.one,
                               inc=0*u.deg, raan=0*u.deg, argp=0*u.deg, nu=0*u.deg)

# Bielliptic transfer (intermediate orbit)
rb = 80000e3
a_bi1 = (r_leo + rb) / 2
ecc_bi1 = (rb - r_leo) / (rb + r_leo)
bielliptic1 = Orbit.from_classical(bodies.Earth, a=a_bi1*u.m, ecc=ecc_bi1*u.one,
                                   inc=0*u.deg, raan=0*u.deg, argp=0*u.deg, nu=0*u.deg)

# Plot comparison
fig, ax = plt.subplots(figsize=(14, 14))

# Earth
earth = plt.Circle((0, 0), 6371, color='lightblue', label='Earth', zorder=5)
ax.add_patch(earth)

# Helper to plot orbit
def plot_orbit_xy(orbit, color, label, style='-', linewidth=2):
    theta = np.linspace(0, 2*np.pi, 200)
    positions = []
    for angle in theta:
        orb = Orbit.from_classical(
            orbit.attractor, orbit.a, orbit.ecc, orbit.inc,
            orbit.raan, orbit.argp, angle * u.rad
        )
        positions.append(orb.r.to_value('km'))
    positions = np.array(positions)
    ax.plot(positions[:, 0], positions[:, 1], style, color=color,
            linewidth=linewidth, label=label, zorder=3)

# Plot orbits
plot_orbit_xy(leo, 'green', 'LEO (Initial)', linewidth=3)
plot_orbit_xy(geo, 'blue', 'GEO (Target)', linewidth=3)
plot_orbit_xy(hohmann, 'red', 'Hohmann Transfer', style='--', linewidth=2.5)
plot_orbit_xy(bielliptic1, 'orange', 'Bielliptic Transfer', style=':', linewidth=2.5)

ax.set_xlabel('X (km)', fontsize=12)
ax.set_ylabel('Y (km)', fontsize=12)
ax.set_title('Transfer Orbit Comparison', fontsize=14, fontweight='bold')
ax.legend(loc='upper right', fontsize=11)
ax.grid(True, alpha=0.3)
ax.set_aspect('equal')
ax.set_xlim(-90000, 90000)
ax.set_ylim(-90000, 90000)
plt.tight_layout()
plt.show()

print("Transfer Orbit Comparison:")
print(f"  Hohmann: 2 burns, {(hohmann.period/2).to(u.hour):.2f} transfer time")
print(f"  Bielliptic: 3 burns, longer transfer time but lower ΔV for large ratios")

## Summary

In this notebook, you learned:

✅ 2D orbit plotting with matplotlib

✅ 3D interactive visualization with plotly

✅ Ground track plotting for satellite coverage

✅ Orbit animations (2D and 3D)

✅ Multiple orbit comparisons

✅ Custom styling and publication-quality plots

✅ Transfer orbit visualization

### Next Steps

Continue with:
- **05_advanced_techniques.ipynb** - Monte Carlo analysis, perturbations, performance optimization

### Tips for Presentations

- Use **plotly** for interactive demos and exploration
- Use **matplotlib** for publication-quality static figures
- Export animations to GIF for easy sharing
- Use dark theme for presentations in dark rooms
- Ground tracks are great for mission coverage analysis